<a href="https://colab.research.google.com/github/bdemchak/cytoscape-jupyter/blob/main/gangsu/basic%20protocol%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a work-in-progress reproduction of the [Biological Network Exploration with Cytoscape 3](https://pubmed.ncbi.nlm.nih.gov/25199793/) Basic Protocol 1, which loads an s. cervesiae network, filters out unneeded nodes, lays out the resulting network, and then creates a dendogram display.

While much of it works, there are compromises, mainly due to Cytoscape features that aren't at full strength yet.

---
# Setup data files, py4cytoscape and Cytoscape connection
---
**NOTE: To run this notebook, you must manually start Cytoscape first -- don't proceed until you have started Cytoscape.**

---
## Setup: Notebook data files

Create the 'output' directory, which will be used to store files uploaded from Cytoscape.

This is a good place to prepare any other system resources that might be needed by downstream Notebook cells.





In [1]:
!rm -r output/
!ls -l 
OUTPUT_DIR = 'output/'

rm: cannot remove 'output/': No such file or directory
total 4
drwxr-xr-x 1 root root 4096 Jun  1 13:40 sample_data


---
## Setup: Fetch latest py4cytoscape




**Note that you can fetch from a specific github branch by adding "@<branch>" to the "py4cytocape" at the end of the github URL.**

For example, to get branch 0.0.5: git+https://github.com/cytoscape/py4cytoscape@0.0.5

In [1]:
!pip uninstall -y py4cytoscape

!pip install py4cytoscape
#!pip install git+https://github.com/cytoscape/py4cytoscape@0.0.9
#!pip install git+https://github.com/cytoscape/py4cytoscape

!pip show py4cytoscape

import py4cytoscape as p4c

Uninstalling py4cytoscape-0.0.9:
  Successfully uninstalled py4cytoscape-0.0.9
  Using cached https://files.pythonhosted.org/packages/e6/4c/b6d4dc780be2de727a3b06c5b99be7290d9d28131691c36361f3ee1fb811/py4cytoscape-0.0.9-py3-none-any.whl
Name: py4cytoscape
Version: 0.0.9
Summary: Cytoscape Automation API
Home-page: https://github.com/cytoscape/py4cytoscape
Author: Barry Demchak
Author-email: bdemchak@ucsd.edu
License: MIT License
Location: /usr/local/lib/python3.7/dist-packages
Requires: requests, colorbrewer, python-igraph, networkx, pandas
Required-by: 


---
## Setup: Set up Cytoscape connection


Set up a "browser client", which is the glue that allows the server-based Notebook to communicate with Cytoscape.

*Note that the IPython.display.Javascript() call must be the last line in this cell.*

In [2]:
import IPython

print(f'Loading Javascript client ... {p4c.get_browser_client_channel()} on {p4c.get_jupyter_bridge_url()}')
browser_client_js = p4c.get_browser_client_js(True)
IPython.display.Javascript(browser_client_js) # Start browser client


Loading Javascript client ... 0ab226bb-8f5d-40c2-997d-5d63d7b5a621 on https://jupyter-bridge.cytoscape.org


<IPython.core.display.Javascript object>

---
# Sanity test to verify Cytoscape connection


By now, the connection to Cytoscape should be up and available. To verify this, try a simple operation that doesn't alter the state of Cytoscape.

In [3]:
p4c.cytoscape_version_info()


{'apiVersion': 'v1',
 'automationAPIVersion': '1.2.0',
 'cytoscapeVersion': '3.9.0-SNAPSHOT',
 'jupyterBridgeVersion': '0.0.2',
 'py4cytoscapeVersion': '0.0.9'}

---
## Setup: Import source data files

The network and annotation files are in a Dropbox folder, and this cell downloads them into the default Sandbox from where Cytoscape will access them.

The files could just as well have been on any cloud resource, including Google Drive, Github, Microsoft OneDrive or a private web site. Note that in this case, the network file was so large that it could not be saved on GitHub, so Dropbox was a handy alternative.

*An alternative would be to load the files into this Notebook file system (or create them there) and then download those files to the Sandbox. Loading them into the Notebook file system would require the use of Notebook "!" commands (e.g., !wget).*

In this cell, we show how the network file can be directly downloaded from Dropbox. The cell that demonstrates gene expression merging (below) shows the use of the "!" command.

**Sandboxing is explained in https://py4cytoscape.readthedocs.io/en/latest/concepts.html#sandboxing**

In [5]:
p4c.sandbox_set(None) # Revert to default sandbox in case some other workflow selected a different one

res_mitab = p4c.sandbox_url_to("https://www.dropbox.com/s/8wc8o897tsxewt1/BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab?dl=0", "BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab")
print(f'Network file BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab has {res_mitab["fileByteCount"]} bytes')

Network file BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab has 166981992 bytes


# Load the s. cerevisiae MITAB network into Cytoscape

Note that the import_network_from_file function (incorrectly) throws an exception, so we explicitly ignore the exception.

**Note**: Once CYTOSCAPE-12772 is fixed, we can remove the try-block in this cell.

In [6]:
from requests import HTTPError
p4c.close_session(False)

try:
  p4c.import_network_from_file('BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab')
except:  
  pass
if p4c.get_network_count() != 1:
  raise Exception('Failed to load network')
net_suid = p4c.get_network_suid()
print(f'Network identifier: {net_suid}')



In commands_post(): {'status': 500, 'type': 'urn:cytoscape:ci:cyrest-core:v1:handle-json-command:errors:3', 'message': 'Task returned invalid json.', 'link': 'file:/C:/Users/CyDeveloper/CytoscapeConfiguration/3/framework-cytoscape.log'}


Network identifier: 7488984


# Merge the gene expression data into the node table

For Cytoscape 3.9.0 and later, call Cytoscape to merge the gene expression data into the node attribute table. 

For pre-Cytoscape 3.9.0, do most of the work in Pandas and then import the dataframe into the node attribute table. Explicitly set the Gene ID as a string even though it's originally parsed as a number. To Cytoscape, the string will be comparable to the 'name' column already in the BIOGRID network. The Gene ID column in the dataframe is matched to the network's name column.



In [7]:
if p4c.check_supported_versions(cytoscape='3.9') is None:
  # Load file directly into Sandbox so Cytoscape can import it
  res_soft = p4c.sandbox_url_to("https://www.dropbox.com/s/r15azh0xb53smu1/GDS112_full.soft?dl=0", "GDS112_full.soft")
  print(f'Annotation file GDS112_full.soft has {res_soft["fileByteCount"]} bytes')

  res = p4c.load_table_data_from_file('GDS112_full.soft', start_load_row=83, data_key_column_index=10, delimiters='\t')
  print(f'Load result contains table identifiers: {res["mappedTables"]}')
else:
  # Load file into Notebook file system so Python can import it, tweak it, and download to Cytoscape
  !wget -q --no-check-certificate https://www.dropbox.com/s/r15azh0xb53smu1/GDS112_full.soft?dl=0
  !mv GDS112_full.soft?dl=0 GDS112_full.soft

  import pandas as df
  GDS112_full = df.read_csv('GDS112_full.soft', skiprows=82, sep='\t')
  GDS112_full.dropna(subset=['Gene ID'], inplace=True)
  GDS112_full['Gene ID'] = df.to_numeric(GDS112_full['Gene ID'], downcast='integer')
  GDS112_full = GDS112_full.astype({'Gene ID': 'string'})
  print(GDS112_full.dtypes)
  print(GDS112_full)
  p4c.load_table_data(GDS112_full, data_key_column='Gene ID')

  import os
  os.remove('GDS112_full.soft')


Annotation file GDS112_full.soft has 5536880 bytes
Load result contains table identifiers: [7488955, 7488993]


# Create a filter to remove nodes having no Gene Symbol

In [8]:
res = p4c.create_column_filter('SymbolOK', 'Gene symbol', '[A-Z0-9]*', 'REGEX')
print(f'Nodes selected: {len(res["nodes"])}')

No edges selected.
Nodes selected: 5495


# Create a subnetwork containing only named nodes

This could take several minutes.

At the end, you should see a view containing all nodes laid out. 

If you see only a single rectangle, it could be that your Cytoscape is set to operate with a small stack size. To increase the stack:

1. terminate Cytoscape

2. a) upgrade Cytoscape to 3.9.0 or later 

  ... or b) use a text editor to add -Xss5M to the cytoscape.vmoptions file in your Cytoscape program directory

3. restart Cytoscape

4. re-run this workflow

In [9]:
new_suid = p4c.create_subnetwork()
print(f'New network identifier: {new_suid}')

New network identifier: 8876882


# Get rid of the original network, which isn't needed anymore

In [10]:
p4c.delete_network(net_suid)
net_suid = new_suid

# Install clusterMaker2 if it hasn't already been installed

In [11]:
p4c.install_app('clusterMaker2')

{}


{}

# Create the hierarchical clustering and dendogram

This returns a large data structure that describes the dendogram.

It also creates a dendogram window that's designed for GUI manipulation. It's unclear this can be controlled or used by automation calls.

**Note:** Having the dendogram is important, and so is having the data that created it. When CSD-420 is addressed, it will be possible to snapshot the dendogram and perform other operations with it.

In [12]:
res = p4c.commands_post('cluster hierarchical showUI=true clusterAttributes=false nodeAttributeList="GSM1029,GSM1030,GSM1032,GSM1033,GSM1034"')
print(f'Dendogram tree: {res}')

Dendogram tree: [{'nodeOrder': [{'nodeName': '850532', 'suid': 7511482}, {'nodeName': '850385', 'suid': 7802491}, {'nodeName': '853043', 'suid': 7517575}, {'nodeName': '853164', 'suid': 7562479}, {'nodeName': '852943', 'suid': 7520494}, {'nodeName': '852397', 'suid': 7506946}, {'nodeName': '853125', 'suid': 7578103}, {'nodeName': '850569', 'suid': 7508473}, {'nodeName': '855726', 'suid': 7499089}, {'nodeName': '853065', 'suid': 7491325}, {'nodeName': '854701', 'suid': 7517161}, {'nodeName': '855825', 'suid': 7589191}, {'nodeName': '855647', 'suid': 7681015}, {'nodeName': '850841', 'suid': 7634692}, {'nodeName': '854344', 'suid': 7495789}, {'nodeName': '850914', 'suid': 7509094}, {'nodeName': '852423', 'suid': 7506211}, {'nodeName': '853771', 'suid': 7600513}, {'nodeName': '853830', 'suid': 7580365}, {'nodeName': '852588', 'suid': 7681984}, {'nodeName': '853163', 'suid': 7638766}, {'nodeName': '852846', 'suid': 7678669}, {'nodeName': '855916', 'suid': 7820653}, {'nodeName': '852467', 's

# Use BiNGO for enrichment analysis

The BiNGO app doesn't have automation entrypoints, so this analysis isn't possible right now. Is there a different app that can do this?

**NOTE:** We need CSD-421 fixed because we don't have any analysis right now, which is very important.